In [1]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [2]:
df_trades = pd.read_pickle("USD_JPY_H4_trades.pkl")

In [3]:
pair = "USD_JPY"

In [4]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair,'M5'))

In [6]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [10]:
df_trades["time"] = [parse(x) for x in df_trades.time]
df_raw["time"] = [parse(x) for x in df_raw.time]


In [12]:
df_trades["next"] = df_trades["time"].shift(-1)

In [22]:
df_trades['trade_end'] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=4)

In [23]:
df_trades[['time', 'next', 'trade_end', 'trade_start']].head()

,time,next,trade_end,trade_start
0,2019-01-02 10:00:00+00:00,2019-01-03 02:00:00+00:00,2019-01-03 05:55:00+00:00,2019-01-02 14:00:00+00:00
1,2019-01-03 02:00:00+00:00,2019-01-03 18:00:00+00:00,2019-01-03 21:55:00+00:00,2019-01-03 06:00:00+00:00
2,2019-01-03 18:00:00+00:00,2019-01-03 22:00:00+00:00,2019-01-04 01:55:00+00:00,2019-01-03 22:00:00+00:00
3,2019-01-03 22:00:00+00:00,2019-01-04 06:00:00+00:00,2019-01-04 09:55:00+00:00,2019-01-04 02:00:00+00:00
4,2019-01-04 06:00:00+00:00,2019-01-04 18:00:00+00:00,2019-01-04 21:55:00+00:00,2019-01-04 10:00:00+00:00


In [ ]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [31]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def process_buy(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price >= TP:
            return 2.0
        elif price <= SL:
            return -1.0
        return 0.0

def process_sell(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price <= TP:
            return 2.0
        elif price >= SL:
            return -1.0
        return 0.0

def process_trade(start_index, direction, TP, SL, prices, start_price):
    if direction == 1:
      return process_buy(start_index, TP, SL, prices, start_price)
    else:
        return process_sell(start_index, TP, SL, prices, start_price)
    

def process_m5(m5_df, row):
    result = 0.0
    for index,price in enumerate(m5_df.mid_c.values):
        if triggered(row.SIGNAL,price, row.ENTRY) == True:
           #print(f"Signal at {index} {price:.2f} {row.ENTRY:.2f} {row.SIGNAL:.2f}")
           result = process_trade(index, row.SIGNAL, row.TAKEPROFIT, row.STOPLOSS, m5_df.mid_c.values,row.ENTRY)
           break 
    return result

In [32]:
total = 0
for index, row in df_trades.iterrows():
     m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
     #print(f"{row.time} {signal_text(row.SIGNAL)} {row.ENTRY:.2f} {m5_data.shape}")
     total += process_m5(m5_data, row)
     #if index > 10:
         #break
print(total)

20.0
